# Conjugados Priores

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Nos capítulos anteriores, utilizámos aproximações de grelha para resolver uma variedade de problemas.

Um dos meus objectivos tem sido mostrar que esta abordagem é suficiente para resolver muitos problemas do mundo real.

E penso que é um bom lugar para começar, porque mostra claramente como os métodos funcionam.

No entanto, como vimos no capítulo anterior, os métodos de grelha só o vão levar até agora.

À medida que aumentamos o número de parâmetros, o número de pontos na grelha cresce (literalmente) exponencialmente.

Com mais de 3-4 parâmetros, os métodos de grelha tornam-se impraticáveis.

Assim, nos restantes três capítulos, apresentarei três alternativas:

1. Neste capítulo vamos utilizar **preliminares conjugados*** para acelerar alguns dos cálculos que já fizemos.

2. No próximo capítulo, vou apresentar os métodos da cadeia de Markov Monte Carlo (MCMC), que podem resolver problemas com dezenas de parâmetros, ou mesmo centenas, num período de tempo razoável.

3. E no último capítulo vamos utilizar a Computação Bayesiana Aproximada (ABC) para problemas que são difíceis de modelar com distribuições simples.

Vamos começar com o problema do Campeonato do Mundo.

## O problema do Campeonato do Mundo Revisitado

Em <<_PoissonProcessos>>, resolvemos o problema do Campeonato do Mundo usando um processo Poisson para modelar os golos num jogo de futebol como eventos aleatórios que são igualmente susceptíveis de ocorrer em qualquer ponto durante um jogo.

Utilizámos uma distribuição gama para representar a distribuição prévia de $\lambda$, a taxa de marcação de golos.  E utilizámos uma distribuição de Poisson para calcular a probabilidade de $k$, o número de golos marcados.

Aqui está um objecto gama que representa a distribuição prévia.

In [4]:
from scipy.stats import gamma

alpha = 1.4
dist = gamma(alpha)

E aqui está uma aproximação da grelha.

In [5]:
import numpy as np
from utils import pmf_from_dist

lams = np.linspace(0, 10, 101)
prior = pmf_from_dist(dist, lams)

Aqui está a probabilidade de marcar 4 golos para cada valor possível de `lam`.

In [6]:
from scipy.stats import poisson

k = 4
likelihood = poisson(lams).pmf(k)

E aqui está a actualização.

In [7]:
posterior = prior * likelihood
posterior.normalize()

Até agora, isto deve ser familiar.

Agora vamos resolver o mesmo problema usando o conjugado anterior.

## O Prior Conjugado

Em <<_TheGammaDistribution>>, apresentei três razões para usar uma distribuição gama para o anterior e disse que havia uma quarta razão que revelaria mais tarde.

Bem, agora é a altura certa.

A outra razão pela qual escolhi a distribuição gama é que é o "conjugado anterior" da distribuição Poisson, assim chamada porque as duas distribuições estão ligadas ou acopladas, que é o que significa "conjugado".

Na secção seguinte explicarei *como* estão ligados, mas primeiro mostrar-vos-ei a consequência desta ligação, que é que existe uma forma notavelmente simples de calcular a distribuição posterior.

Contudo, para o demonstrar, temos de mudar da versão de um parâmetro da distribuição gama para a versão de dois parâmetros.  Uma vez que o primeiro parâmetro se chama "alfa", pode-se adivinhar que o segundo parâmetro se chama "beta".

A seguinte função toma `alfa` e `beta` e faz um objecto que representa uma distribuição gama com esses parâmetros.

In [8]:
def make_gamma_dist(alpha, beta):
    """Makes a gamma object."""
    dist = gamma(alpha, scale=1/beta)
    dist.alpha = alpha
    dist.beta = beta
    return dist

Eis a distribuição prévia com `alpha=1,4` novamente e `beta=1`. 

In [9]:
alpha = 1.4
beta = 1

prior_gamma = make_gamma_dist(alpha, beta)
prior_gamma.mean()

Agora, reivindico sem provas que podemos fazer uma actualização Bayesiana com objectivos `k` apenas fazendo uma distribuição gama com parâmetros `alpha+k` e `beta+1`.

In [10]:
def update_gamma(prior, data):
    """Update a gamma prior."""
    k, t = data
    alpha = prior.alpha + k
    beta = prior.beta + t
    return make_gamma_dist(alpha, beta)

Eis como o actualizamos com `k=4` golos no jogo `t=1`.

In [11]:
data = 4, 1
posterior_gamma = update_gamma(prior_gamma, data)

Depois de todo o trabalho que fizemos com a grelha, pode parecer absurdo que possamos fazer uma actualização Bayesiana, acrescentando dois pares de números.

Portanto, vamos confirmar que funciona.

Vou fazer um `Pmf` com uma discreta aproximação da distribuição posterior.

In [12]:
posterior_conjugate = pmf_from_dist(posterior_gamma, lams)

A figura seguinte mostra o resultado juntamente com o posterior que calculámos utilizando o algoritmo de grelha.

In [13]:
from utils import decorate

def decorate_rate(title=''):
    decorate(xlabel='Goal scoring rate (lam)',
             ylabel='PMF',
             title=title)

In [14]:
posterior.plot(label='grid posterior', color='C1')
posterior_conjugate.plot(label='conjugate posterior', 
                         color='C4', linestyle='dotted')

decorate_rate('Posterior distribution')

São as mesmas, excepto pequenas diferenças devido a aproximações de pontos flutuantes.

In [15]:
np.allclose(posterior, posterior_conjugate)

## O que é o Actual?

Para compreender como isso funciona, escreveremos o PDF do anterior gamma e o PMF da probabilidade de Poisson, depois multiplicá-los juntos, porque é isso que a actualização Bayesiana faz.

Veremos que o resultado é uma distribuição gama, e obteremos os seus parâmetros.

Aqui está o PDF do anterior gamma, que é a densidade de probabilidade para cada valor de $\lambda$, dados os parâmetros $\alpha$ e $\beta$:

$$$lambda^{\\alpha-1} e^$$lambda ^beta

Omiti o factor normalizador; uma vez que estamos a planear normalizar a distribuição posterior de qualquer forma, não precisamos realmente dele.

Agora suponha que uma equipa marca $k$ golos em jogos $t$.

A probabilidade destes dados é dada pelo PMF da distribuição Poisson, que é uma função de $k$ com $\lambda$ e $t$ como parâmetros.

$$lambda^k e^{-lambda t}$$

Mais uma vez, omiti o factor normalizador, o que torna mais claro que as distribuições gama e Poisson têm a mesma forma funcional.

Quando os multiplicamos juntos, podemos emparelhar os factores e somar os expoentes.

O resultado é a distribuição posterior não normalizada,

$$$lambda^{\alpha-1+k} e^{-lambda(beta + t)}$$

que podemos reconhecer como uma distribuição gama não normalizada com parâmetros $\alpha + k$ e $\beta + t$.

Esta derivação fornece uma visão do significado dos parâmetros da distribuição posterior: $\alpha$ reflecte o número de eventos que ocorreram; $\beta$ reflecte o tempo decorrido.

## Probabilidade Binomial

Como segundo exemplo, vejamos novamente o problema do euro.

Quando o resolvemos com um algoritmo de grelha, começámos com um anterior uniforme:

In [16]:
from utils import make_uniform

xs = np.linspace(0, 1, 101)
uniform = make_uniform(xs, 'uniform')

Utilizámos a distribuição binomial para calcular a probabilidade dos dados, que foi de 140 cabeças em 250 tentativas.

In [17]:
from scipy.stats import binom

k, n = 140, 250
xs = uniform.qs
likelihood = binom.pmf(k, n, xs)

Depois calculámos a distribuição posterior da forma habitual.

In [18]:
posterior = uniform * likelihood
posterior.normalize()

Podemos resolver este problema de forma mais eficiente utilizando o conjugado prévio da distribuição binomial, que é a distribuição beta.

A distribuição beta está limitada entre 0 e 1, por isso funciona bem para representar a distribuição de uma probabilidade como `x`.

Tem dois parâmetros, chamados `alfa` e `beta`, que determinam a forma da distribuição.

SciPy fornece um objecto chamado `beta` que representa uma distribuição beta.

A seguinte função toma `alpha` e `beta` e devolve um novo objecto `beta`.

In [19]:
import scipy.stats

def make_beta(alpha, beta):
    """Makes a beta object."""
    dist = scipy.stats.beta(alpha, beta)
    dist.alpha = alpha
    dist.beta = beta
    return dist

Acontece que a distribuição uniforme, que utilizámos anteriormente, é a distribuição beta com os parâmetros `alpha=1` e `beta=1`.

Assim podemos fazer um objecto `beta` que representa uma distribuição uniforme, como este:

In [20]:
alpha = 1
beta = 1

prior_beta = make_beta(alpha, beta)

Agora vamos descobrir como fazer a actualização.  Como no exemplo anterior, vamos escrever o PDF da distribuição anterior e o PMF da função de probabilidade, e multiplicá-los juntos.  Veremos que o produto tem a mesma forma que o anterior, e obteremos os seus parâmetros.

Aqui está o PDF da distribuição beta, que é uma função de $x$ com $\alpha$ e $\beta$ como parâmetros.

$$x^{\alpha-1} (1-x)^{\beta-1}$$

Mais uma vez, omiti o factor normalizador, do qual não precisamos porque vamos normalizar a distribuição após a actualização.

E aqui está o PMF da distribuição binomial, que é uma função de $k$ com $n$ e $x$ como parâmetros.

$$x^{k} (1-x)^{n-k}$$

Mais uma vez, omiti o factor normalizador.

Agora, quando multiplicamos o beta anterior e a probabilidade binomial, o resultado é

$$x^{\\alpha-1+k} (1-x)^{\beta-1+n-k}$$

que reconhecemos como uma distribuição beta não normalizada com parâmetros $\alpha+k$ e $\beta+n-k$.

Assim, se observarmos os sucessos do `k` nos ensaios `n`, podemos fazer a actualização fazendo uma distribuição beta com parâmetros `alpha+k` e `beta+n-k`.

É isso que esta função faz:

In [21]:
def update_beta(prior, data):
    """Update a beta distribution."""
    k, n = data
    alpha = prior.alpha + k
    beta = prior.beta + n - k
    return make_beta(alpha, beta)

Mais uma vez, o anterior conjugado dá-nos uma visão do significado dos parâmetros; $\alpha$ está relacionado com o número de êxitos observados; $\beta$ está relacionado com o número de fracassos.

Eis como fazemos a actualização com os dados observados.

In [22]:
data = 140, 250
posterior_beta = update_beta(prior_beta, data)

Para confirmar que funciona, vou avaliar a distribuição posterior para os possíveis valores de `xs` e colocar os resultados num `Pmf`.

In [23]:
posterior_conjugate = pmf_from_dist(posterior_beta, xs)

E podemos comparar a distribuição posterior que acabámos de calcular com os resultados do algoritmo da grelha.

In [24]:
def decorate_euro(title):
    decorate(xlabel='Proportion of heads (x)',
             ylabel='Probability',
             title=title)

In [25]:
posterior.plot(label='grid posterior', color='C1')
posterior_conjugate.plot(label='conjugate posterior',
                        color='C4', linestyle='dotted')

decorate_euro(title='Posterior distribution of x')

São as mesmas, excepto pequenas diferenças devido a aproximações de pontos flutuantes.

Os exemplos até agora são problemas que já resolvemos, por isso vamos tentar algo novo.

In [26]:
np.allclose(posterior, posterior_conjugate)

## Leões e Tigres e Ursos

Suponhamos que visitamos uma reserva de animais selvagens onde sabemos que os únicos animais são leões, tigres e ursos, mas não sabemos quantos de cada um deles existem.

Durante o passeio, vemos 3 leões, 2 tigres, e um urso. Assumindo que cada animal teve uma oportunidade igual de aparecer na nossa amostra, qual é a probabilidade de que o próximo animal que vemos seja um urso?

Para responder a esta pergunta, vamos utilizar os dados para estimar a prevalência de cada espécie, ou seja, que fracção dos animais pertence a cada espécie.

Se conhecemos as prevalências, podemos utilizar a distribuição multinomial para calcular a probabilidade dos dados.

Por exemplo, suponhamos que sabemos que a fracção de leões, tigres e ursos é de 0,4, 0,3, e 0,3, respectivamente.

Nesse caso, a probabilidade dos dados é:

In [27]:
from scipy.stats import multinomial

data = 3, 2, 1
n = np.sum(data)
ps = 0.4, 0.3, 0.3

multinomial.pmf(data, n, ps)

Agora, poderíamos escolher um prior para as prevalências e fazer uma actualização Bayesiana utilizando a distribuição multinomial para calcular a probabilidade dos dados.

Mas há uma maneira mais fácil, porque a distribuição multinomial tem um conjugado anterior: a distribuição Dirichlet.

## A Distribuição Dirichlet

A distribuição Dirichlet é uma distribuição multivariada, como a distribuição normal multivariada que utilizámos em <<_MultivariadaNormalDistribuição>> para descrever a distribuição das medidas do pinguim.  

Nesse exemplo, as quantidades na distribuição são pares de comprimentos de barbatanas e de culminas, e os parâmetros da distribuição são um vector de meios e uma matriz de covariâncias.

Numa distribuição de Dirichlet, as quantidades são vectores de probabilidades, $\mathbf{x}$, e o parâmetro é um vector, $\mathbf{\alpha}$.

Um exemplo tornará isso mais claro.  SciPy fornece um objecto `dirichlet` que representa uma distribuição Dirichlet.

Aqui está um exemplo com $\mathbf{\alpha} = 1, 2, 3$.

In [28]:
from scipy.stats import dirichlet

alpha = 1, 2, 3
dist = dirichlet(alpha)

Uma vez que fornecemos três parâmetros, o resultado é uma distribuição de três variáveis.

Se retirarmos um valor aleatório desta distribuição, assim:

In [29]:
dist.rvs()

In [30]:
dist.rvs().sum()

O resultado é um conjunto de três valores. 

Estão delimitadas entre 0 e 1, e somam sempre 1, pelo que podem ser interpretadas como as probabilidades de um conjunto de resultados que se excluem mutuamente e são colectivamente exaustivos.

Vamos ver como são as distribuições destes valores.  Desenharei 1000 vectores aleatórios a partir desta distribuição, como esta:

In [31]:
sample = dist.rvs(1000)

In [32]:
sample.shape

O resultado é um conjunto com 1000 filas e três colunas.  Vou calcular o `Cdf` dos valores em cada coluna.

In [33]:
from empiricaldist import Cdf

cdfs = [Cdf.from_seq(col) 
        for col in sample.transpose()]

O resultado é uma lista de objectos `Cdf' que representam as distribuições marginais das três variáveis.  Eis como elas se parecem.

In [34]:
for i, cdf in enumerate(cdfs):
    label = f'Column {i}'
    cdf.plot(label=label)
    
decorate()

A coluna 0, que corresponde ao parâmetro mais baixo, contém as probabilidades mais baixas.

A coluna 2, que corresponde ao parâmetro mais elevado, contém as probabilidades mais elevadas.

Acontece que estas distribuições marginais são distribuições beta.

A seguinte função toma uma sequência de parâmetros, `alfa`, e calcula a distribuição marginal da variável `i`:

In [35]:
def marginal_beta(alpha, i):
    """Compute the ith marginal of a Dirichlet distribution."""
    total = np.sum(alpha)
    return make_beta(alpha[i], total-alpha[i])

Podemos utilizá-lo para calcular a distribuição marginal para as três variáveis.

In [36]:
marginals = [marginal_beta(alpha, i)
             for i in range(len(alpha))]

O gráfico seguinte mostra o CDF destas distribuições como linhas cinzentas e compara-as com os CDFs das amostras.

In [37]:
xs = np.linspace(0, 1, 101)

for i in range(len(alpha)):
    label = f'Column {i}'
    
    pmf = pmf_from_dist(marginals[i], xs)
    pmf.make_cdf().plot(color='C5')
    
    cdf = cdfs[i]
    cdf.plot(label=label, style=':')

decorate()

Isto confirma que os marginais da distribuição Dirichlet são distribuições beta.

E isso é útil porque a distribuição de Dirichlet é o conjugado prévio para a função de probabilidade multinomial.

Se a distribuição anterior é Dirichlet com parâmetro vector `alfa` e os dados são um vector de observações, `dados`, a distribuição posterior é Dirichlet com parâmetro vector `alfa + dados`.

Como exercício no final deste capítulo, pode usar este método para resolver o problema dos Leões e dos Tigres e Ursos.

## Resumo

Depois de ler este capítulo, se sentir que foi enganado, eu compreendo.  Acontece que muitos dos problemas deste livro podem ser resolvidos com apenas algumas operações aritméticas.  Então porque é que nos demos a todo o trabalho de utilizar algoritmos de grelha?

Infelizmente, existem apenas alguns problemas que podemos resolver com priores conjugados; de facto, este capítulo inclui a maioria dos que são úteis na prática.

Para a grande maioria dos problemas, não existe um conjugado prévio e nenhum atalho para calcular a distribuição posterior.

É por isso que precisamos de algoritmos de grelha e dos métodos nos próximos dois capítulos, Computação Bayesiana Aproximada (ABC) e métodos da cadeia de Markov Monte Carlo (MCMC).

## Exercícios


Assim, a probabilidade dos dados é dada pela distribuição exponencial e não pela distribuição Poisson. 

Mas acontece que a distribuição gama é *também* o conjugado prévio da distribuição exponencial, pelo que existe também uma forma simples de calcular esta actualização.

O PDF da distribuição exponencial é uma função de $t$ com $\lambda$ como parâmetro.

$$$lambda e^{-$lambda t}$$

Multiplique o PDF da gama anterior por esta probabilidade, confirme que o resultado é uma distribuição gama não normalizada, e veja se pode derivar os seus parâmetros.

Escrever algumas linhas de código para actualizar `prior_gamma` com os dados desta versão do problema, que foi um primeiro golo após 11 minutos e um segundo golo após 12 minutos adicionais.

Lembre-se de expressar estas quantidades em unidades de jogos, que são aproximadamente 90 minutos.

In [38]:
# Solution goes here

In [39]:
# Solution goes here

In [40]:
# Solution goes here

In [41]:
# Solution goes here

**Exercício:** Para problemas como o problema do Euro onde a função de probabilidade é binomial, podemos fazer uma actualização Bayesiana com apenas algumas operações aritméticas, mas apenas se o anterior for uma distribuição beta.

Se quisermos uma distribuição uniforme prévia, podemos utilizar uma distribuição beta com `alpha=1` e `beta=1`.

Mas o que podemos fazer se a distribuição prévia que queremos não for uma distribuição beta?

Por exemplo, em <<_TrianglePrior>> também resolvemos o problema do Euro com um triângulo anterior, que não é uma distribuição beta.

Nestes casos, podemos frequentemente encontrar uma distribuição beta que é uma aproximação suficiente para o anterior que queremos.

Veja se consegue encontrar uma distribuição beta que se ajuste ao triângulo antes, depois actualize-a utilizando `update_beta`.

Utilizar `pmf_from_dist` para fazer um `Pmf` que se aproxima da distribuição posterior e compará-la com a posterior que acabámos de calcular utilizando um algoritmo de grelha.  Quão grande é a maior diferença entre eles?

Aqui está novamente o triângulo anterior.

In [42]:
from empiricaldist import Pmf

ramp_up = np.arange(50)
ramp_down = np.arange(50, -1, -1)

a = np.append(ramp_up, ramp_down)
xs = uniform.qs

triangle = Pmf(a, xs, name='triangle')
triangle.normalize()

E aqui está a actualização.

In [43]:
k, n = 140, 250
likelihood = binom.pmf(k, n, xs)

posterior = triangle * likelihood
posterior.normalize()

Para começar, aqui está a distribuição beta que utilizámos anteriormente como um uniforme.

In [44]:
alpha = 1
beta = 1

prior_beta = make_beta(alpha, beta)
prior_beta.mean()

E aqui está o que parece em comparação com o triângulo anterior.

In [45]:
prior_pmf = pmf_from_dist(prior_beta, xs)

triangle.plot(label='triangle')
prior_pmf.plot(label='beta')

decorate_euro('Prior distributions')

Agora, é a partir daí.

In [46]:
# Solution goes here

In [47]:
# Solution goes here

In [48]:
# Solution goes here

In [49]:
# Solution goes here

**Exercise:** [3Blue1Brown](https://en.wikipedia.org/wiki/3Blue1Brown) é um canal do YouTube sobre matemática; se ainda não o conhece, recomendo-o vivamente.

Em [this video](https://www.youtube.com/watch?v=8idr1WZ1A7Q) o narrador apresenta este problema:

> Está a comprar um produto online e vê três vendedores a oferecer o mesmo produto ao mesmo preço.  Um deles tem uma avaliação 100% positiva, mas com apenas 10 avaliações.  Outro tem uma avaliação positiva de 96% com 50 avaliações totais.  E ainda outro tem uma avaliação positiva de 93%, mas com um total de 200 opiniões.

>

> A quem se deve comprar?

Vamos pensar em como modelar este cenário.  Suponha que cada vendedor tem alguma probabilidade desconhecida, `x`, de prestar um serviço satisfatório e obter uma classificação positiva, e nós queremos escolher o vendedor com o valor mais alto de `x`.

Este não é o único modelo para este cenário, e não é necessariamente o melhor.  Uma alternativa seria algo como a teoria da resposta ao item, em que os vendedores têm capacidade variável para fornecer um serviço satisfatório e os clientes têm dificuldade variável em ficar satisfeitos.

Mas o primeiro modelo tem a virtude da simplicidade, por isso vamos ver onde nos leva.

1. A priori, sugiro uma distribuição beta com `alpha=8` e `beta=2`.  Como é que se parece este prior e o que é que ele implica para os vendedores?

2. Utilizar os dados para actualizar o anterior para os três vendedores e traçar as distribuições posteriores.  Qual dos vendedores tem a maior média posterior?

3. Até que ponto devemos estar confiantes quanto à nossa escolha?  Ou seja, qual é a probabilidade de o vendedor com a média posterior mais alta ter realmente o valor mais alto de `x`?

4. Considerar um beta anterior com `alfa=0,7` e `beta=0,5`.  Como é este prior e o que implica para os vendedores?

5. Realizar a análise de novo com este prévio e ver que efeito tem sobre os resultados.

Nota: Quando se avalia a distribuição beta, deve-se restringir o intervalo de `xs` para que não inclua 0 e 1. Quando os parâmetros da distribuição beta são inferiores a 1, a densidade de probabilidade vai até ao infinito a 0 e 1. De um ponto de vista matemático, isso não é um problema; continua a ser uma distribuição de probabilidade adequada.  Mas, de um ponto de vista computacional, significa que temos de evitar avaliar o PDF a 0 e 1. 

In [50]:
# Solution goes here

In [51]:
# Solution goes here

In [52]:
# Solution goes here

In [53]:
# Solution goes here

In [54]:
# Solution goes here

In [55]:
# Solution goes here

In [56]:
# Solution goes here

In [57]:
# Solution goes here

**Exercício:** Utilizar um Dirichlet prévio com parâmetro vector `alpha = [1, 1, 1]` para resolver o problema dos Leões e Tigres e Ursos:

>Ponha-se que visitamos uma reserva de animais selvagens onde sabemos que os únicos animais são leões, tigres e ursos, mas não sabemos quantos de cada um deles existem.

>

>Durante o passeio, vemos três leões, dois tigres, e um urso. Assumindo que cada animal teve uma oportunidade igual de aparecer na nossa amostra, estimamos a prevalência de cada espécie.

>

>Qual é a probabilidade de que o próximo animal que vemos seja um urso?



In [58]:
# Solution goes here

In [59]:
# Solution goes here

In [60]:
# Solution goes here

In [61]:
# Solution goes here

In [62]:
# Solution goes here